In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
english_sentences = ['hello', 'how are you', 'good morning', 'thank you', 'bye']
french_sentences = ['bonjour', 'comment ça va', 'bonjour', 'merci', 'au revoir']

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [4]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_sentences)
eng_seqs = eng_tokenizer.texts_to_sequences(english_sentences)
eng_seqs = pad_sequences(eng_seqs, padding='post')

In [5]:
fra_tokenizer = Tokenizer()
fra_tokenizer.fit_on_texts(french_sentences)
fra_seqs = fra_tokenizer.texts_to_sequences(french_sentences)
fra_seqs = pad_sequences(fra_seqs, padding='post')

In [6]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1

In [16]:
input_length=3
embedding_dim = 128
units=64
encoder_inputs = Input(shape=(input_length,))
enc_embed = Embedding(eng_vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h = GRU(units, return_state=True)(enc_embed)

In [17]:
output_length=3
decoder_inputs = Input(shape=(output_length,))
dec_embed = Embedding(fra_vocab_size, embedding_dim)(decoder_inputs)
decoder_gru = GRU(units, return_sequences=True)(dec_embed, initial_state=state_h)
decoder_dense = Dense(fra_vocab_size, activation='softmax')(decoder_gru)

In [18]:
model = Model([encoder_inputs, decoder_inputs], decoder_dense)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 3, 128)         │          1,152 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_4 (Embedding)   │ (None, 3, 128)         │          1,024 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_2 (GRU)               │ [(None, 64), (None,    │         37,248 │ embedding_3[0][0]      │
│                           │ 64)]                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru_3 (GRU)               │ (None, 3, 64)          │         37,248 │ embedding_4[0][0],     │
│                           │                        │                │ gru_2[0][1]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 3, 8)           │            520 │ gru_3[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 77,192 (301.53 KB)

 Trainable params: 77,192 (301.53 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
decoder_target_data = np.expand_dims(np.roll(fra_seqs, -1, axis=1), -1)


In [20]:
model.fit([eng_seqs, fra_seqs], decoder_target_data, batch_size=2, epochs=100, verbose=1)


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.3875 - loss: 2.0726
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5375 - loss: 2.0274
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4625 - loss: 1.9849
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4208 - loss: 1.9413
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4208 - loss: 1.8922
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4208 - loss: 1.8267
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4417 - loss: 1.7467
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5042 - loss: 1.6397
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4625 - loss: 1.5735
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5042 - loss: 1.4101
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3792 - loss: 1.4685 
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5167 - l

In [22]:
loss, accuracy = model.evaluate([eng_seqs, fra_seqs], decoder_target_data)
print(f"\nAccuracy: {accuracy:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - accuracy: 1.0000 - loss: 0.0094

Accuracy: 1.00
